# Markov chains

[Markov chains](https://en.wikipedia.org/wiki/Markov_chain) are systems that evolve in discrete time. The system might have a state that evolves in time and represents the number of particles in a region of space or the probability of a given letter in a language. They are usually described by a state-transition diagram like,

![state-transition diagram](https://upload.wikimedia.org/wikipedia/commons/thumb/2/2b/Markovkate_01.svg/440px-Markovkate_01.svg.png)

which can be described in a matrix like,

$$
T = \left(\begin{matrix}
0.3 & 0.4 \\
0.7 & 0.6
\end{matrix}\right)
$$

We then represent the state of the system like,
$$
x = \left(\begin{matrix}
\text{Quantity in E} \\
\text{Quantity in A}
\end{matrix}\right)
$$

At each time step, we evolve the state by multiplying $ x_{t+1} = T x_t $.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

**Problem:** For the above transition matrix, evolve the matrix according to the above rule and plot the state versus time.

In [4]:
T = np.array([[0.3, 0.4],
              [0.7, 0.6]])

x0 = np.array([1.0, 0.0]).reshape(-1, 1)

# First, second, and third time step evolutions for example,
np.hstack([x0, T.dot(x0), T.dot(T.dot(x0))])

array([[ 1.  ,  0.3 ,  0.37],
       [ 0.  ,  0.7 ,  0.63]])